In [1]:
from new_model import * 

In [2]:
seed = 100
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
#CIFAR 10 : Data 다운로드 경로지정
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#CIFAR 100 : Data 다운로드 경로지정
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def node_prune(model, pruning_ratio):
    for layer in model.modules():
        if isinstance(layer, M_BasicBlock):
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[0][layer.M_relu1.active != 0]
            cutoff1 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu1.active.data[layer.get_2norm()[0] < cutoff1] *= 0
            
            # calculate cutoff value
            nonzero_vals = layer.get_2norm()[1][layer.M_relu2.active != 0]
            cutoff2 = torch.quantile(nonzero_vals, pruning_ratio)

            # set inactive nodes to 0
            layer.M_relu2.active.data[layer.get_2norm()[1] < cutoff2] *= 0

In [5]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

In [6]:
###8. test_model
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

## 모델지정

### -------------------------------------------------- ### 

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MOMENTUM = 0.9

REG = 5e-4

criterion = nn.CrossEntropyLoss()

ratio = 1-0.5**0.19

basic = 2

## cifar10

In [8]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 271.34 sec
Validation loss: 0.0863, Validation accuracy: 0.9728
Epoch 20: progress time is 550.06 sec
Validation loss: 0.0908, Validation accuracy: 0.9712
Epoch 30: progress time is 826.0 sec
Validation loss: 0.0710, Validation accuracy: 0.9780
Epoch 40: progress time is 1099.14 sec
Validation loss: 0.1009, Validation accuracy: 0.9672
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1394.85 sec
Validation loss: 0.1820, Validation accuracy: 0.9452
Epoch 60: progress time is 1692.85 sec
Validation loss: 0.1849, Validation accuracy: 0.9412
Epoch 70: progress time is 1992.95 sec
Validation loss: 0.1083, Validation accuracy: 0.9640
Epoch 80: p

## cifar100

In [8]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_multistepLR_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 265.41 sec
Validation loss: 0.5638, Validation accuracy: 0.8424
Epoch 20: progress time is 540.95 sec
Validation loss: 0.6904, Validation accuracy: 0.8000
Epoch 30: progress time is 845.09 sec
Validation loss: 0.6728, Validation accuracy: 0.8016
Epoch 40: progress time is 1155.78 sec
Validation loss: 0.7938, Validation accuracy: 0.7728
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1466.36 sec
Validation loss: 1.0705, Validation accuracy: 0.7144
Epoch 60: progress time is 1776.72 sec
Validation loss: 1.0671, Validation accuracy: 0.7080
Epoch 70: progress time is 2090.4 sec
Validation loss: 0.7951, Validation accuracy: 0.7740
Epoch 80: p

# Cosine scheduler

In [9]:
## fine_tuning
def fine_tuning(model, mode, EPOCHS, INITIAL_LR):
    # total number of training epochs
    CHECKPOINT_PATH = "./save_seed100_CIFAR100"

    best_val_acc = 0
    current_learning_rate = INITIAL_LR
    
    remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
    print(f'최초 node의 갯수 : {remaining_node1, remaining_node2}')    

    print("==> Training starts!")
    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)    
    for i in range(EPOCHS):
        if i % 40 == 0 :
            print(f'{(i//40)+1}번 자르고 fine tuning을 시작합니다.')
            node_prune(model, ratio)
            
            remaining_node1 = sum([sum(layer.M_relu1.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])
            remaining_node2 = sum([sum(layer.M_relu2.active) for layer in model.modules() if isinstance(layer, M_BasicBlock)])

            print(f'{(i//40)+1}번 pruning 이후 남은 node 는 : {(remaining_node1, remaining_node2)}')
        
        '''
        train loop
        '''            
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0

        train_loss = 0 # track training loss if you want

        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()

            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)

        scheduler.step()    #스케쥴러 사용해보기
            
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc:
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))        
    
        if i % 10 == 9 :  
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                       

    return model

# cifar10

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar10.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_cosine_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 381.35 sec
Validation loss: 0.0985, Validation accuracy: 0.9668
Epoch 20: progress time is 785.44 sec
Validation loss: 0.0882, Validation accuracy: 0.9712
Epoch 30: progress time is 1193.34 sec
Validation loss: 0.0782, Validation accuracy: 0.9732
Epoch 40: progress time is 1518.76 sec
Validation loss: 0.0736, Validation accuracy: 0.9768
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1793.99 sec
Validation loss: 0.1551, Validation accuracy: 0.9548
Epoch 60: progress time is 2070.42 sec
Validation loss: 0.0891, Validation accuracy: 0.9708
Epoch 70: progress time is 2357.0 sec
Validation loss: 0.0882, Validation accuracy: 0.9740
Epoch 80: 

# cifar100

In [10]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## train
    checkpoint = torch.load(f"../1.Wide_ResNet_model/saved_model/WR_28x{basic}_cifar100.pth")
    model.load_state_dict(checkpoint['state_dict'])

    fine_tuning(model, mode=f"28x{basic}_cosine_lr{lr}", EPOCHS=200, INITIAL_LR=lr)

    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 275.31 sec
Validation loss: 0.5431, Validation accuracy: 0.8296
Epoch 20: progress time is 548.91 sec
Validation loss: 0.5398, Validation accuracy: 0.8456
Epoch 30: progress time is 822.28 sec
Validation loss: 0.5873, Validation accuracy: 0.8300
Epoch 40: progress time is 1096.97 sec
Validation loss: 0.5786, Validation accuracy: 0.8344
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1370.48 sec
Validation loss: 0.9046, Validation accuracy: 0.7580
Epoch 60: progress time is 1646.2 sec
Validation loss: 0.8140, Validation accuracy: 0.7724
Epoch 70: progress time is 1922.21 sec
Validation loss: 0.8015, Validation accuracy: 0.7736
Epoch 80: p

## test 결과

### cifar10

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 10).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR10/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9330; Test accuracy: 0.933
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9415; Test accuracy: 0.9415


| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9306; Test accuracy: 0.9306
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 9406; Test accuracy: 0.9406


### cifar100

In [11]:
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_multistepLR_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   
    
for lr in [0.004, 0.02]:
    model = Wide_ResNet(28, basic, 0.3, 100).to(device)        
    ## test
    checkpoint_path = f"./save_seed{seed}_CIFAR100/28x{basic}_cosine_lr{lr}.pth"
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    test_model(model)   

| Wide-Resnet 28x2
Total examples is 10000, correct examples is 6938; Test accuracy: 0.6938
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 7252; Test accuracy: 0.7252
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 7045; Test accuracy: 0.7045
| Wide-Resnet 28x2
Total examples is 10000, correct examples is 7118; Test accuracy: 0.7118


In [9]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = '28x2_CIFAR10_scheduler_cosine_lr_0.1*0.2*0.2', EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:0'), tensor(896., device='cuda:0'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:0'), tensor(784., device='cuda:0'))
Epoch 10: progress time is 231.92 sec
Validation loss: 0.2735, Validation accuracy: 0.9260
Epoch 20: progress time is 471.86 sec
Validation loss: 0.2481, Validation accuracy: 0.9368
Epoch 30: progress time is 712.98 sec
Validation loss: 0.2272, Validation accuracy: 0.9376
Epoch 40: progress time is 953.64 sec
Validation loss: 0.2080, Validation accuracy: 0.9408
41번 자르고 fine tuning을 시작합니다.
41번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:0'), tensor(684., device='cuda:0'))
Epoch 50: progress time is 1194.21 sec
Validation loss: 0.2716, Validation accuracy: 0.9220
Epoch 60: progress time is 1434.89 sec
Validation loss: 0.2116, Validation accuracy: 0.9412
Epoch 70: progress time is 1677.0 sec
Validation loss: 0.2166, Validation accuracy: 0.9372
Epoch 80: progress time is 19

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [15]:
##scheduler 실행 : cosineannealingLR
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:1'), tensor(896., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:1'), tensor(784., device='cuda:1'))
Epoch 10: progress time is 227.21 sec
Validation loss: 1.2041, Validation accuracy: 0.6860
Epoch 20: progress time is 457.96 sec
Validation loss: 1.1663, Validation accuracy: 0.6932
Epoch 30: progress time is 686.22 sec
Validation loss: 1.1738, Validation accuracy: 0.7124
Epoch 40: progress time is 914.2 sec
Validation loss: 1.1939, Validation accuracy: 0.7164
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:1'), tensor(684., device='cuda:1'))
Epoch 50: progress time is 1141.26 sec
Validation loss: 1.2558, Validation accuracy: 0.6704
Epoch 60: progress time is 1368.96 sec
Validation loss: 1.1940, Validation accuracy: 0.7060
Epoch 70: progress time is 1596.41 sec
Validation loss: 1.2019, Validation accuracy: 0.7048
Epoch 80: progress time is 1824

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
##scheduler 실행 : cosine , lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:1'), tensor(896., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:1'), tensor(784., device='cuda:1'))
Epoch 10: progress time is 224.25 sec
Validation loss: 1.2959, Validation accuracy: 0.6480
Epoch 20: progress time is 453.18 sec
Validation loss: 1.2699, Validation accuracy: 0.6584
Epoch 30: progress time is 680.03 sec
Validation loss: 1.2708, Validation accuracy: 0.6808
Epoch 40: progress time is 906.8 sec
Validation loss: 1.2242, Validation accuracy: 0.6876
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:1'), tensor(684., device='cuda:1'))
Epoch 50: progress time is 1133.41 sec
Validation loss: 1.2698, Validation accuracy: 0.6572
Epoch 60: progress time is 1360.16 sec
Validation loss: 1.2401, Validation accuracy: 0.6800
Epoch 70: progress time is 1587.17 sec
Validation loss: 1.1513, Validation accuracy: 0.7116
Epoch 80: progress time is 1816

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

## SEED 10

In [9]:
##CIFAR10
##scheduler 실행 : cosine , lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:1'), tensor(896., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:1'), tensor(784., device='cuda:1'))
Epoch 10: progress time is 221.91 sec
Validation loss: 0.0730, Validation accuracy: 0.9784
Epoch 20: progress time is 450.37 sec
Validation loss: 0.0669, Validation accuracy: 0.9808
Epoch 30: progress time is 678.35 sec
Validation loss: 0.0771, Validation accuracy: 0.9768
Epoch 40: progress time is 905.59 sec
Validation loss: 0.0689, Validation accuracy: 0.9788
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:1'), tensor(684., device='cuda:1'))
Epoch 50: progress time is 1131.92 sec
Validation loss: 0.1272, Validation accuracy: 0.9588
Epoch 60: progress time is 1358.54 sec
Validation loss: 0.1156, Validation accuracy: 0.9668
Epoch 70: progress time is 1585.12 sec
Validation loss: 0.0919, Validation accuracy: 0.9716
Epoch 80: progress time is 181

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [13]:
##CIFAR10
##scheduler 실행 : cosine , lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:1'), tensor(896., device='cuda:1'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:1'), tensor(784., device='cuda:1'))
Epoch 10: progress time is 222.74 sec
Validation loss: 0.2390, Validation accuracy: 0.9212
Epoch 20: progress time is 448.61 sec
Validation loss: 0.2504, Validation accuracy: 0.9184
Epoch 30: progress time is 673.41 sec
Validation loss: 0.2486, Validation accuracy: 0.9216
Epoch 40: progress time is 898.35 sec
Validation loss: 0.2420, Validation accuracy: 0.9240
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:1'), tensor(684., device='cuda:1'))
Epoch 50: progress time is 1122.05 sec
Validation loss: 0.1901, Validation accuracy: 0.9356
Epoch 60: progress time is 1345.25 sec
Validation loss: 0.2101, Validation accuracy: 0.9392
Epoch 70: progress time is 1569.06 sec
Validation loss: 0.2006, Validation accuracy: 0.9460
Epoch 80: progress time is 179

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [13]:
##CIFAR100
##scheduler 실행 : cosine , lr = 0.1*0.2*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:3'), tensor(896., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:3'), tensor(784., device='cuda:3'))
Epoch 10: progress time is 326.75 sec
Validation loss: 0.5154, Validation accuracy: 0.8464
Epoch 20: progress time is 674.67 sec
Validation loss: 0.6502, Validation accuracy: 0.8092
Epoch 30: progress time is 1030.51 sec
Validation loss: 0.6364, Validation accuracy: 0.8136
Epoch 40: progress time is 1386.01 sec
Validation loss: 0.5693, Validation accuracy: 0.8368
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:3'), tensor(684., device='cuda:3'))
Epoch 50: progress time is 1739.93 sec
Validation loss: 0.8348, Validation accuracy: 0.7620
Epoch 60: progress time is 2093.54 sec
Validation loss: 0.7690, Validation accuracy: 0.7800
Epoch 70: progress time is 2466.87 sec
Validation loss: 0.7754, Validation accuracy: 0.7880
Epoch 80: progress time is 2

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [9]:
##CIFAR100
##scheduler 실행 : cosine , lr = 0.1*0.2
fine_tuning(model, mode = mode, EPOCHS = 200)

최초 node의 갯수 : (tensor(896., device='cuda:3'), tensor(896., device='cuda:3'))
==> Training starts!
1번 자르고 fine tuning을 시작합니다.
1번 pruning 이후 남은 node 는 : (tensor(784., device='cuda:3'), tensor(784., device='cuda:3'))
Epoch 10: progress time is 409.77 sec
Validation loss: 1.0880, Validation accuracy: 0.6972
Epoch 20: progress time is 767.85 sec
Validation loss: 1.0676, Validation accuracy: 0.6940
Epoch 30: progress time is 1029.9 sec
Validation loss: 1.1193, Validation accuracy: 0.7008
Epoch 40: progress time is 1292.82 sec
Validation loss: 1.1541, Validation accuracy: 0.7064
2번 자르고 fine tuning을 시작합니다.
2번 pruning 이후 남은 node 는 : (tensor(684., device='cuda:3'), tensor(684., device='cuda:3'))
Epoch 50: progress time is 1554.18 sec
Validation loss: 1.1701, Validation accuracy: 0.6824
Epoch 60: progress time is 1815.97 sec
Validation loss: 1.1280, Validation accuracy: 0.7076
Epoch 70: progress time is 2077.89 sec
Validation loss: 1.0236, Validation accuracy: 0.7324
Epoch 80: progress time is 23

Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

## 결과 확인

In [11]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_cosine.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9428; Test accuracy: 0.9428


In [10]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_cosine_lr_0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9428; Test accuracy: 0.9428


In [12]:
## cifar10
check_prune = torch.load("./saved_tuning/28x2_CIFAR10_scheduler_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_CIFAR10_scheduler_cosine finished')

Total examples is 10000, correct examples is 9433; Test accuracy: 0.9433


## CIFAR100

In [16]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x2_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

send_email('28x2_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7158; Test accuracy: 0.7158


In [10]:
## cifar100
check_prune = torch.load("./saved_tuning_CIFAR100/28x2_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_cosine_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7079; Test accuracy: 0.7079


## SEED 10

In [10]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x2_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9350; Test accuracy: 0.935


In [14]:
## cifar10
check_prune = torch.load("./save_seed10_CIFAR10/28x2_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])

test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 9422; Test accuracy: 0.9422


In [14]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x2_cosine_lr0.1*0.2*0.2.pth")

model.load_state_dict(check_prune['state_dict'])
test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7067; Test accuracy: 0.7067


In [10]:
## cifar100
check_prune = torch.load("./save_seed10_CIFAR100/28x2_cosine_lr0.1*0.2.pth")

model.load_state_dict(check_prune['state_dict'])
test_model(model)

# send_email('28x2_multistepLR_lr0.1*0.2*0.2 finished')

Total examples is 10000, correct examples is 7143; Test accuracy: 0.7143
